In [1]:
# Initiate GPU runtime

import tensorflow as tf
tf.test.gpu_device_name()

import torch
torch.cuda.get_device_name(0)

'Tesla T4'

In [2]:
# Install Dorado (Oxford Nanopore Technologies)

%%shell

DORADO_VERSION=0.9.0 # Updated 21 May 2025

wget https://cdn.oxfordnanoportal.com/software/analysis/dorado-"$DORADO_VERSION"-linux-x64.tar.gz -O dorado.tar.gz
tar -vzxf dorado.tar.gz; rm -rf dorado.tar.gz; mv dorado-"$DORADO_VERSION"-linux-x64 ont-dorado
mkdir -p ont-dorado/models
./ont-dorado/bin/dorado download --directory ont-dorado/models
./ont-dorado/bin/dorado --version

--2025-05-21 15:06:22--  https://cdn.oxfordnanoportal.com/software/analysis/dorado-0.9.0-linux-x64.tar.gz
Resolving cdn.oxfordnanoportal.com (cdn.oxfordnanoportal.com)... 108.138.94.114, 108.138.94.29, 108.138.94.96, ...
Connecting to cdn.oxfordnanoportal.com (cdn.oxfordnanoportal.com)|108.138.94.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2593394059 (2.4G) [application/x-tar]
Saving to: ‘dorado.tar.gz’

dorado.tar.gz       100%[===================>]   2.42G  85.6MB/s    in 14s     

2025-05-21 15:06:37 (173 MB/s) - ‘dorado.tar.gz’ saved [2593394059/2593394059]

dorado-0.9.0-linux-x64/bin/
dorado-0.9.0-linux-x64/bin/dorado
dorado-0.9.0-linux-x64/lib/
dorado-0.9.0-linux-x64/lib/libnvrtc.so.11.2
dorado-0.9.0-linux-x64/lib/libdorado_torch_lib.so
dorado-0.9.0-linux-x64/lib/libnvToolsExt.so.1
dorado-0.9.0-linux-x64/lib/libsz.so.2.0.1
dorado-0.9.0-linux-x64/lib/libnvrtc.so.11.8.89
dorado-0.9.0-linux-x64/lib/libhdf5.so.8
dorado-0.9.0-linux-x64/lib/libnvToolsE

In [3]:
# Install MegaTools

!sudo apt-get install -y megatools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  megatools
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 207 kB of archives.
After this operation, 898 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 megatools amd64 1.10.3-1build1 [207 kB]
Fetched 207 kB in 0s (486 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package megatools.
(Reading database ... 126102 files and directorie

In [5]:
# Download FAST5/POD5 files

%%shell

FILES="https://mega.nz/file/NU113SpT#5JwWToDmQrt4yV8wWgpz8_oMYjB4CI6JV9Agek8d8K8"

megadl "$FILES" --path /content/

Downloaded GraphPad Prism v9.0.0.121 - WINDOWS.zip


In [ ]:
# Basecall and Demultiplex process

%%shell

# Specify the name of the folder (i.e., 20240411_1608_MN22990_FAT84733_f175dd69/fast5)
DIR="20240411_1608_MN22990_FAT84733_f175dd69/fast5"

# Specify the barcode kit used for library preparation
BARKIT="EXP-NBD114"

PREFIX="$(echo "$IDIR" | awk -F/ '{print $1}')"

pod5 convert fast5 $IDIR --output "/content/gdrive/MyDrive/"$PREFIX".pod5"

# Specify the accurate basecalling model - fast (fast), high (hac) or super (sup) accuracy
# dna_r9.4.1_e8_fast@v3.4
# dna_r9.4.1_e8_hac@v3.3
# dna_r9.4.1_e8_sup@v3.3
./ont-dorado/bin/dorado basecaller "ont-dorado/models/dna_r9.4.1_e8_sup@v3.3" \
"/content/gdrive/MyDrive/"$PREFIX".pod5" -r --min-qscore 8 --emit-fastq >> "/content/gdrive/MyDrive/"$PREFIX"_DORADO-BASECALL.fastq"

./ont-dorado/bin/dorado demux "/content/gdrive/MyDrive/"$PREFIX"_DORADO-SUP.fastq" --emit-fastq \
--output-dir "/content/gdrive/MyDrive/"$PREFIX"_DORADO-DEMUX" --kit-name $BARKIT -t $(nproc)